# Training Classification Models

In [1]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

c:\Users\user\Deep Learning\melanoma-segmentation-and-classification\venv\lib\site-packages\albumentations\__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.16). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
metadata_train = pd.read_csv("../data/metadata/ISIC-2017_Training_Data.csv")
metadata_validation = pd.read_csv("../data/metadata/ISIC-2017_Validation_Data.csv")
base_dir = "data"
image_folder_train= "ISIC-2017_Training_Data"
gt_folder_train="ISIC-2017_Training_Part1_GroundTruth"
image_folder_val= "ISIC-2017_Validation_Data"
gt_folder_val="ISIC-2017_Validation_Part1_GroundTruth"

img_pt_train = os.path.join(base_dir, image_folder_train)
gt_pt_train = os.path.join(base_dir, gt_folder_train)
img_pt_val = os.path.join(base_dir, image_folder_val)
gt_pt_val = os.path.join(base_dir, gt_folder_val)

metadata_train.head()

,image_id,melanoma,age_approximate,sex
0,ISIC_0000000,0,55,female
1,ISIC_0000001,0,30,female
2,ISIC_0000002,1,60,female
3,ISIC_0000003,0,30,male
4,ISIC_0000004,1,80,male


In [3]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Define augmentations
train_aug = A.Compose([
    A.RandomResizedCrop(224, 224),
    A.HorizontalFlip(p=0.5),                 # Flip horizontally
    A.Rotate(limit=30, p=0.5),               # Rotate within ±30 degrees
    A.ColorJitter(brightness=0.2, contrast=0.2, p=0.5),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()                             # Convert to Tensor
], additional_targets={'mask': 'mask'})

val_aug = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])
#The numbers in mean and std are the mean and standard deviation of the ImageNet dataset.
#The dimensions of the images are 224x224 pixels. This is the input size of the pre-trained models in PyTorch.
# Apply augmentations in Dataset

class MelanomaDatasetClassification(Dataset):
    def __init__(self, metadata, img_dir, mask_dir, augmentations=None):
        self.metadata = metadata
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.augmentations = augmentations

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        row = self.metadata.iloc[idx]
        image_id = row['image_id']
        label = row['melanoma']

        # Load the image
        img_path = os.path.join(self.img_dir, f"{image_id}.jpg")
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB

        # Load the mask
        mask_path = os.path.join(self.mask_dir, f"{image_id}_segmentation.png")
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)  # Load mask as grayscale

        # Apply augmentations (if provided)
        if self.augmentations:
            augmented = self.augmentations(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask, label

In [4]:
# Train Dataset with augmentations
train_dataset = MelanomaDatasetClassification(metadata_train, img_pt_train,gt_pt_train, augmentations=train_aug)

# Validation Dataset without augmentations
val_dataset = MelanomaDatasetClassification(metadata_validation, img_pt_val,gt_pt_val, augmentations=val_aug)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim

# Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10, device='cuda'):
    model.to(device)

    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        print("-" * 30)

        # Training phase
        model.train()
        train_loss = 0.0
        correct_train = 0
        total_train = 0

        for images, masks, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device).float().unsqueeze(1)  # Adjust label shape for binary classification

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Track training loss and accuracy
            train_loss += loss.item() * images.size(0)
            preds = (outputs > 0.5).float()
            correct_train += (preds == labels).sum().item()
            total_train += labels.size(0)

        train_loss /= total_train
        train_accuracy = correct_train / total_train

        # Validation phase
        model.eval()
        val_loss = 0.0
        correct_val = 0
        total_val = 0

        with torch.no_grad():
            for images, masks, labels in val_loader:
                images = images.to(device)
                labels = labels.to(device).float().unsqueeze(1)

                # Forward pass
                outputs = model(images)
                loss = criterion(outputs, labels)

                # Track validation loss and accuracy
                val_loss += loss.item() * images.size(0)
                preds = (outputs > 0.5).float()
                correct_val += (preds == labels).sum().item()
                total_val += labels.size(0)

        val_loss /= total_val
        val_accuracy = correct_val / total_val

        print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")
        print(f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")
        print()

    return model


In [6]:
#CUDA enable if not cpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


## ResNet18



In [7]:
from torchvision import models

# Load pretrained ResNet18
resnet18 = models.resnet18(pretrained=True)
# Modify the final fully connected layer
num_features = resnet18.fc.in_features
resnet18.fc = nn.Sequential(
    nn.Linear(num_features, 1),  # Binary classification
    nn.Sigmoid()                 # Output probabilities
)

# Loss function and optimizer
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(resnet18.parameters(), lr=0.001)

# Train ResNet18
trained_resnet18 = train_model(resnet18, train_loader, val_loader, criterion, optimizer, num_epochs=10)


c:\Users\user\Deep Learning\melanoma-segmentation-and-classification\venv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\user\Deep Learning\melanoma-segmentation-and-classification\venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


AssertionError: Torch not compiled with CUDA enabled

## EfficientNet-B0

In [ ]:
from torchvision.models import efficientnet_b0

# Load pretrained EfficientNet-B0
effnet = efficientnet_b0(pretrained=True)

# Modify the classifier
num_features = effnet.classifier[1].in_features
effnet.classifier = nn.Sequential(
    nn.Linear(num_features, 1),
    nn.Sigmoid()
)

# Loss function and optimizer
optimizer = optim.Adam(effnet.parameters(), lr=0.001)

# Train EfficientNet-B0
trained_effnet = train_model(effnet, train_loader, val_loader, criterion, optimizer, num_epochs=10)


## MobileNetV2

In [ ]:
from torchvision.models import mobilenet_v2

# Load pretrained MobileNetV2
mobilenet = mobilenet_v2(pretrained=True)

# Modify the classifier
num_features = mobilenet.last_channel
mobilenet.classifier = nn.Sequential(
    nn.Linear(num_features, 1),
    nn.Sigmoid()
)

# Loss function and optimizer
optimizer = optim.Adam(mobilenet.parameters(), lr=0.001)

# Train MobileNetV2
trained_mobilenet = train_model(mobilenet, train_loader, val_loader, criterion, optimizer, num_epochs=10)


In [ ]:
torch.save(trained_resnet18.state_dict(), "resnet18_melanoma.pth")
torch.save(trained_effnet.state_dict(), "effnet_melanoma.pth")
torch.save(trained_mobilenet.state_dict(), "mobilenet_melanoma.pth")